In [429]:
import pandas as pd
import numpy as np
import torch
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn import functional as F
import pickle
from torch.utils.data import DataLoader, TensorDataset
import ast

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import gensim.downloader

from torch.distributions import Categorical

from tokenizers import Tokenizer

type_dict={
    0:'Creature',
    1:'Sorcery',
    2:'Artifact',
    3:'Enchantment',
    4:'Instant',
    5:'Land',
}

In [415]:
# For ZY no csv read
#with open('mtgdata.pickle', 'rb') as file:
#    mtg_df=pickle.load(file)
mtg_df=pd.read_csv('mtg_data.csv', index_col=0)
mtg_df=mtg_df.dropna(subset=['text_prompt', 'card_description'])
mtg_df.head()

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,rarity,flavor_text,text,text_prompt,card_description
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],uncommon,"""A rift opened, and our arrows were abruptly s...",Fury Sliver: [SEP] {5}{R} [SEP] Creature — Sli...,Fury Sliver: [SEP] {5}{R},Creature — Sliver [SEP] All Sliver creatures h...
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When ~ enters the battlefield, you may attach ...",2,2,['W'],['W'],[],common,"""We take only what we need to survive. Believe...",Kor Outfitter: [SEP] {W}{W} [SEP] Creature — K...,Kor Outfitter: [SEP] {W}{W},Creature — Kor Soldier [SEP] When ~ enters the...
2,Spirit,NaN,0.0,Token Creature — Spirit,Flying,1,1,['W'],['W'],[Flying],common,NaN,Spirit: [SEP] [SEP] Token Creature — Spirit [...,Spirit: [SEP],Token Creature — Spirit [SEP] Flying
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,"Flying\nWhen ~ enters the battlefield, it expl...",1,2,['U'],['U'],"[Flying, Explore]",common,NaN,Siren Lookout: [SEP] {2}{U} [SEP] Creature — S...,Siren Lookout: [SEP] {2}{U},Creature — Siren Pirate [SEP] Flying\nWhen ~ e...
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,['G'],['G'],[Enchant],rare,NaN,Web: [SEP] {G} [SEP] Enchantment — Aura [SEP] ...,Web: [SEP] {G},Enchantment — Aura [SEP] Enchant creature (Tar...


In [416]:
#load pretrained embeddings
wiki_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [417]:
def generate_embedding(phrase):
    words = word_tokenize(phrase)
    words = [word.lower() for word in words]
    lemmatizer=WordNetLemmatizer()
    #stemmer=PorterStemmer()
    words = [lemmatizer.lemmatize(word) for word in words]
    #words = [stemmer.stem(word) for word in words]
    total_vector=[]
    for word in words:
        try:
            total_vector.append(wiki_vectors.word_vec(word))
        except KeyError:
            pass
    if len(total_vector)!=0:
        out = np.mean(total_vector, axis=0)
    else:
        out = np.zeros(50)
    return out

In [418]:
types=['Creature', 'Sorcery', 'Artifact', 'Enchantment', 'Instant', 'Land']
type_list={'Name':[],
           'Type':[]}
for idx, row in mtg_df.iterrows():
    for word in row['type_line'].split():
        if word in types:
            type_list['Name'].append(row['name'])
            type_list['Type'].append(word)
type_data=pd.DataFrame(type_list).drop_duplicates()
for type in types:
    type_data[type]=type_data['Type']==type
type_data

,Name,Type,Creature,Sorcery,Artifact,Enchantment,Instant,Land
0,Fury Sliver,Creature,True,False,False,False,False,False
1,Kor Outfitter,Creature,True,False,False,False,False,False
2,Spirit,Creature,True,False,False,False,False,False
3,Siren Lookout,Creature,True,False,False,False,False,False
4,Web,Enchantment,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
83968,Born to Drive,Enchantment,False,False,False,True,False,False
83978,Stern Mentor,Creature,True,False,False,False,False,False
83981,Youthful Valkyrie,Creature,True,False,False,False,False,False
83982,Fallaji Vanguard,Creature,True,False,False,False,False,False


In [419]:
total_embeddings=[]
for idx, row in type_data.iterrows():
    total_embeddings.append(generate_embedding(row['Name']))

tensor_list = [torch.tensor(nameembedding) for nameembedding in list(total_embeddings)]
name_embeddings = torch.stack(tensor_list, dim=0)
name_embeddings

C:\Users\hammerhao\AppData\Local\Temp\ipykernel_27156\1671889651.py:11: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  total_vector.append(wiki_vectors.word_vec(word))


tensor([[ 0.1918, -0.0154,  0.1995,  ..., -0.2442, -0.3086, -0.6314],
        [-0.4758,  1.0394, -0.2369,  ...,  0.2503,  0.1288,  0.0442],
        [-0.0175,  0.7970, -1.3675,  ..., -0.3775, -0.2226, -0.3518],
        ...,
        [ 0.4284,  0.2326, -0.1057,  ..., -0.2599,  0.0209,  0.2470],
        [ 0.4418, -0.1677,  0.5107,  ...,  0.3453, -0.3805,  0.7088],
        [ 0.3030,  0.8218, -0.1229,  ..., -0.3748, -0.0187, -0.1198]],
       dtype=torch.float64)

In [420]:
card_type=torch.tensor(type_data.drop(['Name', 'Type'], axis=1).values, dtype=torch.long)
n_train = int(0.9*type_data.shape[0])
train_X = name_embeddings[:n_train]
val_X = name_embeddings[n_train:]
train_Y = card_type[:n_train]
val_Y = card_type[n_train:]

In [421]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

# Define the input size, hidden layer size, and number of classes
input_size = 50
hidden_size = 128
num_classes = 6

# Create an instance of the MLPClassifier
model = MLPClassifier(input_size, hidden_size, num_classes)

# You can print the model to see its architecture
print(model)

MLPClassifier(
  (fc1): Linear(in_features=50, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=6, bias=True)
  (softmax): Softmax(dim=-1)
)


In [422]:
num_epochs=100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_X=train_X.float()
train_Y=train_Y.float()
dataset=TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)

for epoch in range(num_epochs):
    total_loss = 0.0

    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader)}')

print('Training finished.')
torch.save(model.state_dict(), 'type.pt')

Epoch [1/100], Loss: 1.5123923901979088
Epoch [2/100], Loss: 1.456056068693926
Epoch [3/100], Loss: 1.4482141657529144
Epoch [4/100], Loss: 1.4432151686721646
Epoch [5/100], Loss: 1.4393703480662428
Epoch [6/100], Loss: 1.4313410508451123
Epoch [7/100], Loss: 1.4248880970901645
Epoch [8/100], Loss: 1.4214295569410178
Epoch [9/100], Loss: 1.4185308163541221
Epoch [10/100], Loss: 1.4158060278384212
Epoch [11/100], Loss: 1.4132201892470346
Epoch [12/100], Loss: 1.4107968719477582
Epoch [13/100], Loss: 1.408813845990273
Epoch [14/100], Loss: 1.4067678136873971
Epoch [15/100], Loss: 1.4050778894860128
Epoch [16/100], Loss: 1.4033012843979191
Epoch [17/100], Loss: 1.401336113813565
Epoch [18/100], Loss: 1.399466518218142
Epoch [19/100], Loss: 1.3979771778668244
Epoch [20/100], Loss: 1.3963851946864636
Epoch [21/100], Loss: 1.3949075664965633
Epoch [22/100], Loss: 1.3933066734807746
Epoch [23/100], Loss: 1.3916807089965355
Epoch [24/100], Loss: 1.3905276217436426
Epoch [25/100], Loss: 1.38887

In [423]:
def generate_type(cardname):
    probs = model(torch.tensor(generate_embedding(cardname)))
    distribution = Categorical(probs)
    sampled_index = distribution.sample()
    for key, type in type_dict.items():
        print(f'{type}:{probs[key]}')
    return type_dict[int(sampled_index)]

In [424]:
generate_type('sword')

Creature:1.4916816819867828e-19
Sorcery:8.51060851548339e-19
Artifact:1.0
Enchantment:5.450759930084514e-15
Instant:3.8833258830806017e-10
Land:5.295954059290725e-29


C:\Users\hammerhao\AppData\Local\Temp\ipykernel_27156\1671889651.py:11: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  total_vector.append(wiki_vectors.word_vec(word))


'Artifact'

In [412]:
colors_wheel=['W', 'U', 'R', 'G', 'B', 'E']
color_list={'Name':[],
           'color':[]}
for idx, row in mtg_df.iterrows():
    colors = ast.literal_eval(row['color_identity'])
    if len(colors)!=0:
        for color in colors:
            color_list['Name'].append(row['name'])
            color_list['color'].append(color)
    else:
            color_list['Name'].append(row['name'])
            color_list['color'].append('E')
color_data=pd.DataFrame(color_list).drop_duplicates()
for color in colors_wheel:
     color_data[color]=color_data['color']==color
color_data

,Name,color,W,U,R,G,B,E
0,Fury Sliver,R,False,False,True,False,False,False
1,Kor Outfitter,W,True,False,False,False,False,False
2,Spirit,W,True,False,False,False,False,False
3,Siren Lookout,U,False,True,False,False,False,False
4,Web,G,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
99030,Youthful Valkyrie,W,True,False,False,False,False,False
99031,Fallaji Vanguard,R,False,False,True,False,False,False
99032,Fallaji Vanguard,W,True,False,False,False,False,False
99035,Hold at Bay,W,True,False,False,False,False,False


In [399]:
total_embeddings=[]
for idx, row in color_data.iterrows():
    total_embeddings.append(generate_embedding(row['Name']))
tensor_list = [torch.tensor(nameembedding) for nameembedding in list(total_embeddings)]
name_embeddings = torch.stack(tensor_list, dim=0)

card_color=torch.tensor(color_data.drop(['Name', 'color'], axis=1).values, dtype=torch.long)
n_train = int(0.9*color_data.shape[0])
color_train_X = name_embeddings[:n_train]
color_val_X = name_embeddings[n_train:]
color_train_Y = card_color[:n_train]
color_val_Y = card_color[n_train:]

C:\Users\hammerhao\AppData\Local\Temp\ipykernel_27156\1671889651.py:11: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  total_vector.append(wiki_vectors.word_vec(word))


In [400]:
# Define the input size, hidden layer size, and number of classes
input_size = 50
hidden_size = 128
num_classes = 6

class MLPColorClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPColorClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out

# Create an instance of the MLPClassifier
color_model = MLPColorClassifier(input_size, hidden_size, num_classes)

In [401]:
num_epochs=300

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(color_model.parameters(), lr=1e-3)
color_train_X=color_train_X.float()
color_train_Y=color_train_Y.float()
dataset=TensorDataset(color_train_X, color_train_Y)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)

for epoch in range(num_epochs):
    total_loss = 0.0

    for inputs, labels in train_loader:
        # Forward pass
        outputs = color_model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader)}')

print('Training finished.')
torch.save(color_model.state_dict(), 'color.pt')

Epoch [1/300], Loss: 1.7164844825703611
Epoch [2/300], Loss: 1.682784531449759
Epoch [3/300], Loss: 1.6722558636819163
Epoch [4/300], Loss: 1.6635801902381322
Epoch [5/300], Loss: 1.6562638785249444
Epoch [6/300], Loss: 1.6498920943147393
Epoch [7/300], Loss: 1.6441362006689912
Epoch [8/300], Loss: 1.6371911502653553
Epoch [9/300], Loss: 1.6324077852310672
Epoch [10/300], Loss: 1.6267303538578812
Epoch [11/300], Loss: 1.62289163707405
Epoch [12/300], Loss: 1.6187378706470612
Epoch [13/300], Loss: 1.6155278213562505
Epoch [14/300], Loss: 1.6114462960150935
Epoch [15/300], Loss: 1.6070230250717492
Epoch [16/300], Loss: 1.6048936651599022
Epoch [17/300], Loss: 1.6013630877258958
Epoch [18/300], Loss: 1.5985366939216532
Epoch [19/300], Loss: 1.5956242581849458
Epoch [20/300], Loss: 1.5935455878575644
Epoch [21/300], Loss: 1.5906789592517319
Epoch [22/300], Loss: 1.5882746873363371
Epoch [23/300], Loss: 1.5863361253533312
Epoch [24/300], Loss: 1.583976787136447
Epoch [25/300], Loss: 1.58116

In [402]:
color_dict={
    0:'W',
    1:'U',
    2:'R',
    3:'G',
    4:'B',
    5:'Colorless',
}

def generate_color(cardname):
    probs = color_model(torch.tensor(generate_embedding(cardname)))
    distribution = Categorical(probs)
    sampled_index = distribution.sample()
    for key, type in color_dict.items():
        print(f'{type}:{probs[key]}')
    return color_dict[int(sampled_index)]

In [427]:
mtg_df=mtg_df.dropna(subset='oracle_text')
mtg_df.head(5)

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,rarity,flavor_text,text,text_prompt,card_description
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],uncommon,"""A rift opened, and our arrows were abruptly s...",Fury Sliver: [SEP] {5}{R} [SEP] Creature — Sli...,Fury Sliver: [SEP] {5}{R},Creature — Sliver [SEP] All Sliver creatures h...
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When ~ enters the battlefield, you may attach ...",2,2,['W'],['W'],[],common,"""We take only what we need to survive. Believe...",Kor Outfitter: [SEP] {W}{W} [SEP] Creature — K...,Kor Outfitter: [SEP] {W}{W},Creature — Kor Soldier [SEP] When ~ enters the...
2,Spirit,NaN,0.0,Token Creature — Spirit,Flying,1,1,['W'],['W'],[Flying],common,NaN,Spirit: [SEP] [SEP] Token Creature — Spirit [...,Spirit: [SEP],Token Creature — Spirit [SEP] Flying
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,"Flying\nWhen ~ enters the battlefield, it expl...",1,2,['U'],['U'],"[Flying, Explore]",common,NaN,Siren Lookout: [SEP] {2}{U} [SEP] Creature — S...,Siren Lookout: [SEP] {2}{U},Creature — Siren Pirate [SEP] Flying\nWhen ~ e...
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,['G'],['G'],[Enchant],rare,NaN,Web: [SEP] {G} [SEP] Enchantment — Aura [SEP] ...,Web: [SEP] {G},Enchantment — Aura [SEP] Enchant creature (Tar...


In [443]:
types=['Creature', 'Sorcery', 'Artifact', 'Enchantment', 'Instant', 'Land']
colors_wheel=['W', 'U', 'R', 'G', 'B', 'E']
type_color_list={'Name':[],
                 'Type':[],
                 'Color':[],
                 'oracle_text':[]}
for idx, row in mtg_df.iterrows():
    colors = ast.literal_eval(row['color_identity'])
    for word in row['type_line'].split():
        if word in types:
            if len(colors)!=0:
                for color in colors:
                    type_color_list['Name'].append(row['name'])
                    type_color_list['Type'].append(word)
                    type_color_list['Color'].append(color)
                    type_color_list['oracle_text'].append(row['oracle_text'])
            else:
                type_color_list['Name'].append(row['name'])
                type_color_list['Type'].append(word)
                type_color_list['Color'].append('E')
                type_color_list['oracle_text'].append(row['oracle_text'])

type_color_data=pd.DataFrame(type_color_list)
type_color_data

,Name,Type,Color,oracle_text
0,Fury Sliver,Creature,R,All Sliver creatures have double strike.
1,Kor Outfitter,Creature,W,"When ~ enters the battlefield, you may attach ..."
2,Spirit,Creature,W,Flying
3,Siren Lookout,Creature,U,"Flying\nWhen ~ enters the battlefield, it expl..."
4,Web,Enchantment,G,Enchant creature (Target a creature as you cas...
...,...,...,...,...
96994,Angel's Tomb,Artifact,E,Whenever a creature enters the battlefield und...
96995,Horned Troll,Creature,G,{G}: Regenerate ~.
96996,Faerie Bladecrafter,Creature,B,Flying\nWhenever one or more Faeries you contr...
96997,Exultant Skymarcher,Creature,W,Flying


In [444]:
rare_char={
    '¡®°²½˝̶π’„•…™−∞☐œŠ':'',
    'Äàáâãä':'a',
    'Éèéêë':'e',
    'Ææ':'ae',
    'Óóö':'o',
    'úûü':'u',
    'íī':'i',
    'Ññ':'n'
}
for rarechar, target in rare_char.items():
    for char in [*rarechar]:
        type_color_data['oracle_text']=type_color_data['oracle_text'].str.replace(char, target)

In [445]:
tokenizer = Tokenizer.from_file("mtggenerator_tokenizer_v6.json")
vocab_size=tokenizer.get_vocab_size()
encode = lambda text: tokenizer.encode(text).ids
decode = lambda list: tokenizer.decode(list)

In [447]:
text_list=list(type_color_data['oracle_text'])
#convert data to 2d tensor
encoded_text_list=[torch.Tensor(encode(text)) for text in text_list]
max_len=max([len(item) for item in encoded_text_list])
padded_text_list=[torch.cat((item, torch.full((max_len - len(item),), 3))) for item in encoded_text_list] # the [PAD] token has id=3
padded_text_list_with_CLS = [torch.cat((torch.tensor([1]), item)) for item in padded_text_list]

data = pad_sequence(padded_text_list_with_CLS, batch_first=True).long()

In [466]:
for type in types:
    type_color_data[type]=type_color_data['Type']==type
for color in colors_wheel:
     type_color_data[color]=type_color_data['Color']==color
type_color_encodings = torch.tensor(type_color_data.drop(['Name', 'Type', 'Color', 'oracle_text'], axis=1).values, dtype=torch.long)

In [467]:
type_color_encodings.shape

torch.Size([96999, 12])

In [468]:
data.shape

torch.Size([96999, 150])